In [1]:
#%matplotlib nbagg

import geofield
import dipole, fibonacci_sphere
import scipy, numpy
import newleg
import xyzfield
from matplotlib import pyplot, rcParams
import matplotlib
rcParams.update({'font.family': "Source Sans Pro"})
rcParams.update({'font.size': 14})

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import bootstrap
import importlib; importlib.reload(bootstrap)

<module 'bootstrap' from '/home/josecper/Programs/python/bootstrap.py'>

In [3]:
sha=geofield.SHADIFData("../data/sha.dif.14k/coef_EPSL_tnmgh.dat", "../data/sha.dif.14k/error_coef_EPSL_tnmgh.dat")

In [4]:
(x,y,z), (x_dev, y_dev, z_dev)=bootstrap.bootstrap(dipole.ecc_dipole, (sha.gcomp[... , :8],), errors=(sha.gcomp_err[..., :8],))

In [5]:
pyplot.show(pyplot.plot(sha.years, z,
                        sha.years, z+z_dev,
                        sha.years, z-z_dev))

In [4]:
def szrphi(x,y,z):
    s, zz, phi = dipole.xyz2cyl(x,y,z)
    r = numpy.sqrt(x**2 + y**2 + z**2)
    return s, zz, r, phi

#wow this is cool as heck
bigfunc = lambda g: szrphi(*dipole.ecc_dipole(g))

In [42]:
(s,z,r, phi), (s_dev, z_dev, r_dev, phi_dev) = bootstrap.bootstrap(bigfunc, (sha.gcomp[..., :8],), errors=(sha.gcomp_err[..., :8],))

In [5]:
fig, ax = pyplot.subplots(figsize=(20,10))

ax.plot(sha.years, s, color="green", label="s")
ax.plot(sha.years, z, "m-", label="z")
ax.plot(sha.years, r, "b-", label="r")

ax.fill_between(sha.years, r-r_dev, r+r_dev, color="blue", lw=0, alpha=0.07)
ax.fill_between(sha.years, z-z_dev, z+z_dev, color="magenta", lw=0, alpha=0.07)
ax.fill_between(sha.years, s-s_dev, s+s_dev, color="green", lw=0, alpha=0.07)

ax.set_xlabel("Years AD")
ax.set_ylabel("s,z,r (km)")
ax.legend()

ax.set_xbound(-1000,sha.years[-1])

pyplot.show(fig)

NameError: name 's' is not defined

In [21]:
#this is fuckingf expensive my dude, better run it on a BIG COMPUTE
!date
(nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev) = bootstrap.bootstrap(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=1, order=1, south=False)
!date
#other things to bootstrap:
# - the rest of the poles (potentially forever)
# - average intensity (mildly expensive, just take less points)
# - pqpd (not expensive)
# - curvature (just re-bootstrap from the pole calculations)
# - paris curve

lun jun 20 13:33:11 CEST 2016
Failed to save <class 'numpy.ndarray'> to .npy file:
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/joblib/numpy_pickle.py", line 271, in save
    obj, filename = self._write_array(obj, filename)
  File "/usr/lib/python3.5/site-packages/joblib/numpy_pickle.py", line 231, in _write_array
    self.np.save(filename, array)
  File "/usr/lib/python3.5/site-packages/numpy/lib/npyio.py", line 477, in save
    fid = open(file, "wb")
OSError: [Errno 28] No queda espacio en el dispositivo: '/tmp/joblib/joblib/newleg/legendre/fd97c87232f22c2f2a5d88a85edea521/output.pkl_02.npy'

Failed to save <class 'numpy.ndarray'> to .npy file:
Traceback (most recent call last):
  File "/usr/lib/python3.5/site-packages/joblib/numpy_pickle.py", line 271, in save
    obj, filename = self._write_array(obj, filename)
  File "/usr/lib/python3.5/site-packages/joblib/numpy_pickle.py", line 231, in _write_array
    self.np.save(filename, array)
  File "/usr/lib

In [24]:
fig, (ax1, ax2) = pyplot.subplots(2,1)

ax1.plot(sha.years, nlon_1, "b-")
ax1.plot(sha.years, nlon_1+nlon_1_dev, "b--")
ax1.plot(sha.years, nlon_1-nlon_1_dev, "b--")

ax2.plot(sha.years, nlat_1, "b-")
ax2.plot(sha.years, nlat_1+nlat_1_dev, "b--")
ax2.plot(sha.years, nlat_1-nlat_1_dev, "b--")

pyplot.show(fig)

In [6]:
from joblib import Parallel, delayed

#be carfeful my man
def boot_it(order):
    return bootstrap.bootstrap(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=1, order=order, south=False)

orders = (1, 2, 4, 10)
(((nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev)),
 ((nlon_2, nlat_2), (nlon_2_dev, nlat_2_dev)),
 ((nlon_4, nlat_4), (nlon_4_dev, nlat_4_dev)),
 ((nlon_10, nlat_10), (nlon_10_dev, nlat_10_dev))) = Parallel(n_jobs=4)(delayed(boot_it)(o) for o in orders)

### get urself 2400 coffees before hitting shift+enter my dude

In [ ]:
#hack bootstrap to use minimal differences on the stdev calculation?

((nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=300, n_jobs=4, order=1, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_1.txt", numpy.array((nlon_1, nlat_1, nlon_1_dev, nlat_1_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")
((nlon_2, nlat_2), (nlon_2_dev, nlat_2_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=300, n_jobs=4, order=2, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_2.txt", numpy.array((nlon_2, nlat_2, nlon_2_dev, nlat_2_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")
((nlon_4, nlat_4), (nlon_4_dev, nlat_4_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=400, n_jobs=4, order=4, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_4.txt", numpy.array((nlon_4, nlat_4, nlon_4_dev, nlat_4_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")
((nlon_10, nlat_10), (nlon_10_dev, nlat_10_dev)) = bootstrap.parallelize(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=400, n_jobs=4, order=10, south=False, debug=True)
numpy.savetxt("../data/pole_drift/deg_10.txt", numpy.array((nlon_10, nlat_10, nlon_10_dev, nlat_10_dev)).T, header="#Magnetic pole drift data (SHA.DIF.14k) (lon, lat, lon_dev, lat_dev) in degrees")

In [4]:
theta, phi = fibonacci_sphere.grid(4000)

def avg_intensity(gcomp):    
    intensities = numpy.empty_like(sha.years)
    npoints = len(phi)

    for i, year in enumerate(sha.years):
        x,y,z = xyzfield.xyzfieldv2(gcomp[i, :], phi, theta)
        dec,inc,f,h = xyzfield.xyz2difh(x,y,z)
        intensities[i]=(f.sum(axis=0)/npoints)
        
    return intensities

In [ ]:
(intensities, intensities_dev) = bootstrap.parallelize(avg_intensity, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=300, n_jobs=4, debug=True)

0:01:17.773983 | iterations: 8 / 300

In [ ]:
def 

p_dip=dipole.power(sha.gcomp, lmin=1, lmax=1)
p_quad=dipole.power(sha.gcomp, lmin=2, lmax=2)